<a href="https://colab.research.google.com/github/polymoe/datascientest/blob/main/Light_01_python_rnn_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://datascientest.fr/train/assets/logo_datascientest.png" style="height:150px">

<hr style="border-width:2px;border-color:#75DFC1">
<h1 style = "text-align:center" > Réseau de neurones récurrents </h1>


Ce notebook est destiné à pratiquer les notions évoquées dans le premier exercice du module sur une machine plus adaptée.

Si c'est la première fois que vous utilisez colab, n'hésitez pas à jeter un coup d'oeil sur ce [notebook](https://colab.research.google.com/drive/1jXEKOk3mRYBqFWoVwJ0ZpsRJCWj46Yxt?usp=sharing).



In [1]:
# Importation des données

!wget https://train-exo.s3.eu-west-1.amazonaws.com/675/shakespeare.txt

--2022-07-02 19:36:19--  https://train-exo.s3.eu-west-1.amazonaws.com/675/shakespeare.txt
Resolving train-exo.s3.eu-west-1.amazonaws.com (train-exo.s3.eu-west-1.amazonaws.com)... 52.218.85.32
Connecting to train-exo.s3.eu-west-1.amazonaws.com (train-exo.s3.eu-west-1.amazonaws.com)|52.218.85.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>]   1.06M  1.02MB/s    in 1.0s    

2022-07-02 19:36:21 (1.02 MB/s) - ‘shakespeare.txt’ saved [1115394/1115394]



Un des interêts principaux de colab est la mise à disposition d'un GPU. Utiliser un GPU permet d'accelerer grandement l'execution et donc l'entrainement de modèle de deep learning. Pour configurer le GPU (processeur graphique), il suffit de cliquer sur Edit > Notebook settings et sélectionner GPU comme accélérateur matériel.
* Exécuter la cellule suivante pour vérifier que le GPU soit bien activé.

In [2]:
import tensorflow as tf 
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please change your hardware accelerator")

Default GPU Device:/device:GPU:0


# Mise en Pratique


* Ouvrir et lire le fichier shakespeare.txt avec un encoding 'utf-8'. Stocker le fichier dans la variable text.
* Afficher les 250 premiers caractères.



In [3]:
## Insérez votre code ici



In [4]:
#@title Solution
with open("shakespeare.txt", "r", encoding='utf-8') as file:
    text = file.read()
print(text[:250])



First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



* Stocker dans la variable **vocab** tous les caractères uniques du texte. Trier le résultat à l'aide de `sorted`.


* Afficher le nombre de caractère unique.


<div class="alert alert-info">
<i class="fa fa-info-circle"></i> &emsp; 
La commande `set` permet de retourner les éléments uniques d'une liste ou d'une chaine de caractère.
</div>

In [5]:
## Insérez votre code ici



In [6]:
#@title Solution
vocab = sorted(set(text))
print('{} uniques characters'.format(len(vocab)))

65 uniques characters


### Convertir en index

> Comme dans les tâches traditionnelles de text mining, les algorithmes ne traitent que des nombres et non des données textuelles. Il est alors nécessaire de convertir chaque caractère de notre corpus en nombre (ici, des indexes).

* Créer un dictionnaire **char2idx**, dans le lequel vous allez stocker le **caractère en key** et **son index en value** :

In [7]:
## Insérez votre code ici



In [8]:
#@title Solution
char2idx = {u:i for i, u in enumerate(vocab)}

* Exécuter la cellule suivante pour afficher les premiers éléments du dictionnaire **`char2idx`**.

In [9]:
import numpy as np
print('{')
for char, _ in zip(char2idx, range(10)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ...
}


* Convertir dans la liste **`text_as_int`** tous les caractères du texte en index.

In [10]:
## Insérez votre code ici



In [11]:
#@title Solution
text_as_int = [char2idx[c] for c in text]

## Génération du Dataset

Nous allons implémenter dans la suite l'approche prédisant le caractère suivant de chaque élément en entrée (many to many). Nous allons choisir des séquences de caractère en entrée de taille 100.


* Importer **`tensorflow`** sous le nom **`tf`**.


* Créer la variable **seq_length** = 100 représentant le nombre de caractère par séquence.


* Définir un dataset **`char_dataset`** à l'aide de la fonction `from_tensor_slices` du jeu de données **`text_as_int`**.

In [12]:
## Insérez votre code ici



In [13]:
#@title Solution
import tensorflow as tf
seq_length = 100

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

> Nous allons maintenant appliquer une succession de transformation pour mettre notre jeu de données en forme :
>
><img src='https://datascientest.fr/train/assets/tensorflow_04_dataset.jpg' style='width:600px'>


* À l'aide de la méthode `batch` appliquée à **`char_dataset`**, découper le jeu de données en batch de longueur **`seq_length + 1`**. Préciser en argument de la méthode **`drop_remainder = True`** indiquant que le dernier batch doit être supprimé dans le cas où il est incomplet. Stocker le résultat sous le nom **`sequences`**.

In [14]:
## Insérez votre code ici



In [15]:
#@title Solution
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

* Exécuter la cellule suivante pour afficher sous forme de caractères les 2 premiers éléments de **`char_dataset`**.

In [16]:
idx2char = np.array(vocab)
for item in sequences.take(2):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


* À l'aide de la méthode `map` de **`char_dataset`**, séparer les données d'entrées et les données cibles. Stocker le résultat sous le nom **`dataset`**.

In [17]:
## Insérez votre code ici



In [18]:
#@title Solution
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

* Exécuter la cellule suivante pour afficher sous forme de caractères l'input et la target du premier élément de **dataset**.

* Appliquer la méthode `shuffle` à **dataset** en précisant un **buffer_size** de 10000 pour mélanger le jeu de données.


* Appliquer la méthode `batch` à **dataset** en précisant un **batch_size** de 64 pour séparer le jeu de données en batch. Préciser en argument de la méthode **drop_remainder = True** (ne pas prendre le dernier lot incomplet).

In [19]:
## Insérez votre code ici



In [20]:
#@title Solution
batch_size = 64
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

<hr style="border-width:2px;border-color:#75DFC1">
<h2 style = "text-align:center" > Modélisation </h2>
<hr style="border-width:2px;border-color:#75DFC1">

>Dans la partie précédente, nous avons mis en forme notre jeu de données. Nous allons maintenant implémenter notre modèle RNN :
>
* Définir la variable **`vocab_size`** étant le nombre d'élément de notre dictionnaire.


* Définir dans une fonction `build_model` avec comme argument **`batch_size`**:

> * Instancier un modèle séquentiel **`model`** à l'aide du constructeur `Sequential` de **`tensorflow.keras`**.
>
>
> * Définir une couche embedding `Embedding` en précisant une entrée de taille **`vocab_size`** et une sortie de taille 256. Par ailleurs, comme dans la suite nous allons utiliser **`stateful = True`** dans le RNN, il est nécessaire de préciser l'argument **`batch_input_shape = [batch_size, None]`**. 
>
>
> * Ajouter au **`model`** une couche `RNN` avec une cellule `GRUCell` de dimension 512 retournant une séquence et réutilisant le dernier état comme état initial $h_0$. 
>
>
> * Ajouter au **`model`** une couche `Dense` avec **vocab_size** neurones et une fonction d'activation `'softmax'`.
>
>
> * Retourner le **`model`**.

In [21]:
## Insérez votre code ici



In [22]:
#@title Solution
from tensorflow.keras.layers import RNN, GRUCell, Dense, Embedding

# Length of the vocabulary in chars
vocab_size = len(vocab)

def build_model(batch_size):

    model = tf.keras.Sequential()

    model.add(Embedding(vocab_size, 256,
                         batch_input_shape=[batch_size, None]))

    model.add(RNN(GRUCell(512), # Cell of RNN
                return_sequences=True, # return a sequence
                stateful=True))

    model.add(Dense(vocab_size, activation='softmax'))
    
    return model

* Définir un modèle sous le nom **model** à l'aide de la fonction `build_model` avec un batch_size de 64.


* Compiler le modèle à l'aide de sa méthode `compile` en précisant l'optimiseur `Adam` avec un learning rate de **1e-3** et la fonction de perte `sparse_categorical_crossentropy`.


* Afficher le résumé du modèle.

In [23]:
## Insérez votre code ici



In [24]:
#@title Solution
# Create model
model = build_model(64)
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='sparse_categorical_crossentropy')
# Summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 rnn (RNN)                   (64, None, 512)           1182720   
                                                                 
 dense (Dense)               (64, None, 65)            33345     
                                                                 
Total params: 1,232,705
Trainable params: 1,232,705
Non-trainable params: 0
_________________________________________________________________


* Entraîner le modèle sur les données **dataset** sur 30 epochs à l'aide de la méthode `fit`.

In [25]:
## Insérez votre code ici



In [26]:
#@title Solution
model.fit(dataset, epochs=30)

Epoch 1/30
172/172 [==============================] - 21s 102ms/step - loss: 2.5975
Epoch 2/30
172/172 [==============================] - 18s 100ms/step - loss: 1.9940
Epoch 3/30
172/172 [==============================] - 18s 100ms/step - loss: 1.7453
Epoch 4/30
172/172 [==============================] - 18s 100ms/step - loss: 1.6059
Epoch 5/30
172/172 [==============================] - 19s 105ms/step - loss: 1.5206
Epoch 6/30
172/172 [==============================] - 19s 101ms/step - loss: 1.4643
Epoch 7/30
172/172 [==============================] - 20s 112ms/step - loss: 1.4237
Epoch 8/30
172/172 [==============================] - 22s 108ms/step - loss: 1.3913
Epoch 9/30
172/172 [==============================] - 19s 101ms/step - loss: 1.3653
Epoch 10/30
172/172 [==============================] - 18s 100ms/step - loss: 1.3442
Epoch 11/30
172/172 [==============================] - 18s 100ms/step - loss: 1.3241
Epoch 12/30
172/172 [==============================] - 19s 104ms/step - lo

> Ici, le nombre de batch est fixé, il n'est alors pas possible de générer des textes avec des batchs autres que 64. Pour contourner cette limitation, c'est-à-dire traiter un unique texte, on peut utiliser l'astuce suivante :
>
> * Sauvegarder les poids de notre modèle à l'aide de la méthode `save_weights`.
>
>
> * Recharger un nouveau modèle avec un batch_size de 1.
>
>
> * Charger les poids de l'ancien modèle à l'aide de la méthode `load_weights`.


* Sauvegarder les poids du modèle à l'aide de la méthode `save_weights` en précisant le chemin **"model_rnn.h5"**.


* Créer un nouveau modèle sous le nom **`model`** à l'aide de la fonction `build_model` en précisant un batch_size de 1.


* Charger les poids de notre modèle entraîné à l'aide de la méthode `load_weights` en précisant le chemin **"model_rnn.h5"**.

In [27]:
## Insérez votre code ici



In [28]:
#@title Solution
# Save weights
model.save_weights('model_rnn.h5')
# Create a new model with a batch_size of 1.
model = build_model(1)
# Load weights
model.load_weights('model_rnn.h5')

* Exécuter la cellule suivante pour générer un texte commençant par **"ROMEO: "**.

In [29]:
def generate_text(model, start_string, num_generate = 500):
    # Converting our start string to index (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    # Simulate a batch of 1 element
    input_eval = tf.expand_dims(input_eval, 0)
    # List contains the text generated
    text_generated = []
    # Reset initial state
    model.reset_states()
    for i in range(num_generate):
        # Probability prediction
        prediction = model(input_eval)
        # Index prediction
        index = tf.argmax(prediction, axis=-1).numpy()[0]
        input_eval = tf.expand_dims([index[-1]], 0)
        # Save letter in text_generated list
        text_generated.append(idx2char[index[-1]])
    # Return all sequence
    return (start_string + ''.join(text_generated))

print(generate_text(model, start_string="ROMEO:"))

ROMEO:
He is a word with you, and let them gave thee strike.

GRUMIO:
A boy, the matter, the conquest thousand men
That thou art a word with the sacrament was the world to thee,
That we will be the sun that the death of the world they are not so fair,
So many thousand men are no lesser than the life
To bear the senate was the matter, he shall not be so fair.

HENRY BOLINGBROKE:
What is the stroke of it. A shadow lies me with the season which he has been broke,
And so I think, that I may be contented:



> Dans l'ensemble de l'exercice, vous avez formé un réseau de neurones pour **générer du texte**. Même si la longueur du texte que nous échantillonnons est assez petite, nous pouvons néanmoins remarquer certaines caractéristiques intéressantes du texte généré. Par exemple, le réseau apprend quelques mots de base comme «and», «of» «you» et «could» assez rapidement dans la formation. 
>
> En outre, il apprend également quelques règles grammaticales de base: mettre des majuscules au premier mot de la phrase, finir une phrase par de la ponctuation, poser des questions fonctionne si les phrases commencent par des mots comme «what», «where». Le système n'est pas parfait mais étant donné que nous avons alimenté le réseau en caractères simples, il est remarquable que le réseau soit capable d'apprendre ces dépendances à long terme.


<hr style="border-width:2px;border-color:#75DFC1">
<h2 style = "text-align:center" > Ce qu'il faut retenir </h2> 
<hr style="border-width:2px;border-color:#75DFC1">

### Quand utiliser les RNN ?

> Les couches *récurrentes* sont utilisées lorsque les données d'entrées sont séquentielles et que la prédiction du prochain élément de la suite dépend fortement de la prédiction effectuée sur les éléments précédents. Typiquement, les cas où les couches récurrentes sont les plus utilisées sont:
>
> * Une suite de mots arrangés séquentiellement, c'est-à-dire un texte.
>
>
> * Une suite de fréquences, c'est-à-dire un son ou une série temporelle.
>
>
> * Une suite d'images, c'est-à-dire une vidéo.

### Représentation vectoriel des caractères

> L'entrée du modèle est alimentée par une séquence d'indice de caractère. Comme pour les variables catégorielles, il est nécessaire de les transformer sous forme d'un vecteur (get_dumnies, one hot, embedding ...) :
>
> * Couche `Embedding` (présenté dans le prochain exercice) :
>
> ```python
>tf.keras.layers.Embedding(input_dim=vocab_size,       # l'indice maximal + 1 (taille du vocabulaire)
>                           output_dim=embedding_dim)   # taille du vecteur en sortie 
>```
>
>
> * Représentation `one_hot` : Il n'existe pas de layer **one hot** dans le package Keras. Pour ce faire, la fonction `Lambda` de **`tensorflow.keras.layers`** permet de créer un layer personnalisée avec comme argument la fonction à appliquer et la forme de sortie **`output_shape`**.
>
>```python
layer_one_hot = Lambda(lambda x: tf.one_hot(x, depth), output_shape=[depth])
>```


### Entrée/sortie RNN :

> Il existe différentes catégories d'entrée/sortie pour les RNN :
>
> * **Many to One** : Prédire une valeur de sortie $t_n$ en fonction de la séquence d'entrée $t_{n-1}$ ... $t_0$.
>
> <img src='https://assets-datascientest.s3-eu-west-1.amazonaws.com/notebooks/python-deeplnp-text-tm-many-to-one.png' style='width:600px'>
>
>
> * **Many to Many** : Prédire une séquence [$t_n$ ... $t_1$] en fonction de la séquence d'entrée $t_{n-1}$ ... $t_0$.
>
> <img src='https://assets-datascientest.s3-eu-west-1.amazonaws.com/notebooks/python-deeplnp-text-tm-many-to-many.png' style='width:600px'>
>
>
> * **Many to Many** : Prédire une séquence [$t_n$ ... $t_{i}$] en fonction de la séquence d'entrée $t_{i-1}$ ... $t_0$.
>
> <img src='https://assets-datascientest.s3-eu-west-1.amazonaws.com/notebooks/python-deeplnp-text-tm-many-to-many V2.png' style='width:600px'>


### Cellule de calcul RNN

>Il existe différentes cellules de calcul RNN (`SimpleRNNCell`, `LSTMCell` et `GRUCell`). Cette notion sera étudiée plus en profondeur dans les prochains exercices.
>
><img src='https://datascientest.fr/train/assets/tensorflow_04_rnn_gru_lstm.png' style='width:700px'>
>
><center><b>RNN Cell</b></center>